# Backtesting

This notebook will perform the backtesting of our SVM model on our dataset. 

We will calculate the total profit/loss of our strategy, as well as information about the trading frequency, the profit per trade, and other information about the performance of our strategy. 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as ticker
import datetime

import math
import scipy.stats
import sklearn
from sklearn import preprocessing
from sklearn import svm

%load_ext autoreload
%autoreload 2

In [2]:
# Load in dataset
df_gg = pd.read_csv('./data/df_gg_processed.csv')
df_ggl = pd.read_csv('./data/df_ggl_processed.csv')

# Read in train-test fold data
info = np.load('./data/info.npy')

In [3]:
def create_pl_df(df_ticker1, df_ticker2, info):
    """
    Creates a comprehensive DataFrame by combining the ticker data from the given tickers 
    with the residual information calculated in the OU class. 

    Saves this DataFrame to self.df_residual. 
    
    ## TODO: Rewrite GOOG/GOOGL Labels to make more abstract. 

    :df_ticker1: Ticker data from first asset. 
    :df_ticker2: Ticker data from second asset. 
    :info: Fold dictionary calculated from OU class. 

    """
    ticker1_name = df_ticker1['TICKER']
    ticker2_name = df_ticker2['TICKER']
    
    df_test_only = pd.DataFrame()
    df_test_only_labels = pd.Series()

    for i in info:
        idx = i['test']['index']
        residuals = i['test']['residuals_transform_price']
        beta = i['train']['beta_fit_price']
        df_temp = pd.concat([df_ticker1.loc[idx]['CLOSE'], 
                             beta * df_ticker2.loc[idx]['CLOSE'], 
                             df_ticker1.loc[idx]['price'], 
                             beta * df_ticker2.loc[idx]['price'], 
                             i['test']['residuals_transform_price'], 
                             df_ticker1.loc[idx]['TIMESTAMP']
                            ], axis=1)
        df_test_only = df_test_only.append(df_temp)
        df_test_only_labels = df_test_only_labels.append(i['test']['labels'])

    df_test_only['label'] = df_test_only_labels
    df_test_only.columns=['GOOG', 'beta*GOOGL', 'GOOG_return', 
                          'beta*GOOGL_return', 'residual', 'TIMESTAMP', 'label']
    # Find Profit of last trade given percent returns. 
    # (y-(1-deltaY)y) - (x-(1-deltaX)x)
    df_test_only['beta*GOOGL_gains'] = df_test_only['beta*GOOGL']-(1-df_test_only['beta*GOOGL_return'])*df_test_only['beta*GOOGL']
    df_test_only['GOOG_gains'] = df_test_only['GOOG']-(1-df_test_only['GOOG_return'])*df_test_only['GOOG']
    df_test_only['profit'] = df_test_only['beta*GOOGL_gains'] - df_test_only['GOOG_gains']
    df_test_only['TIMESTAMP'] = pd.to_datetime(df_test_only.TIMESTAMP)
    print('P/L Dataframe saved as "df_residual" parameter.')
    return df_test_only

In [4]:
def param_format(params):
    """
    Function that formats a dictionary of parameters into a string that is 
    writeable to a file/directory. 
    
    :params: Parameter Dictionary to format. 
    
    :ret: Stringified version of the parameter dictionary. 
    """
    param_str = ', '.join("{!s}-{!r}".format(key,val) for (key,val) in params.items())
    param_str = param_str.replace("{", "")
    param_str = param_str.replace("}", "")
    param_str = param_str.replace("'", "")
    param_str = param_str.replace(",", "_")
    param_str = param_str.replace(" ", "")
    param_str = param_str.replace(":", "")
    param_str = param_str.replace(".", "")
    param_str = param_str.strip()
    
    return param_str

In [5]:
def sharpe(df_temp):
    """
    Takes in a profit/loss dataframe with labels and finds the Sharpe Ratio according to 
    the following formula: 
    
    Daily P/L / Standard Dev(Daily P/L) x sqrt(252)
    
    :df_temp: Profit/loss dataframe. 
    
    :ret: Sharpe Ratio.
    """
    daily_profit = df_temp.groupby(by = 
                                   [df_temp.TIMESTAMP.dt.year, 
                                    df_temp.TIMESTAMP.dt.month, 
                                    df_temp.TIMESTAMP.dt.day])['profit_timeline'].sum()
    
    return np.mean(daily_profit)/np.std(daily_profit) * np.sqrt(252)


def sortino(df_temp):
    """
    Takes in a profit/loss dataframe with labels and finds the Sortino Ratio according to 
    the following formula: 
    
    Daily P/L / Standard Dev(Negative Daily P/L) x sqrt(252)
    
    :df_temp: Profit/loss dataframe. 
    
    :ret: Sortino Ratio.
    """
    daily_profit = df_temp.groupby(by = 
                                   [df_temp.TIMESTAMP.dt.year, 
                                    df_temp.TIMESTAMP.dt.month, 
                                    df_temp.TIMESTAMP.dt.day])['profit_timeline'].sum()
    
    return np.mean(daily_profit)/np.std([pl for pl in daily_profit if pl < 0]) * np.sqrt(252)
    

def precision(pred, label):
    return (pred[np.logical_and(pred==1, label==1)].shape[0])/pred[pred==1].shape[0]


def fit(params, info):
    """
    Fit a SVM according to the parameters across all folds in the info dictionary. 
    Returns a flattened numpy array of prediction labels across all folds. 

    Adds the labels for the parameter to the residual graph. 

    :params: Model Hyperparameters.
    :info_dict: Associated Info Dict.

    :ret: Numpy array of predictions.
    """
    preds = []

    for i in info:
        b_svm = svm.SVC(**params)
        b_svm.fit(i['train']['df_scale'], i['train']['labels'])
        pred = b_svm.predict(i['test']['df_scale'])
        preds.append(pred)
        
    return np.hstack(np.array(preds))


def find_profit_loss(df_test, preds, params, window=5, threshold=0.0005, plot=True, save_dir=None):
    """
    Performs P/L Backtesting given some prediction labels and a test dataframe. 

    Returns a dictionary of results. 
    
    :df_test: DataFrame of profit loss. 
    :preds: Prediction Labels: 
    :params: Name of Params of model tested. 
    :window: Evaluation window for how long we want to hold on to our position. 
    :threshold: how much of a residual shift determines a trade being offloaded. 
    :plot: Flag for evaluation plots. 
    :save_dir: Folder to save evaluation plots. 
    
    :ret: results. 
    """
    backtest_results = {}
    # Turn params into string for file output
    param_str = param_format(params)
    print("Finding Profit/Loss for model with parameters: %s." % param_str)
    total_profit = 0.0
    
    # Daily profit p/l
    profit_timeline = []
    # Per trade p/l
    trade_profit_timeline = []
    # Amount of minutes held per trade
    time_held_timeline = []
    # Trades held portfolio
    trades_data = []

    df_temp = df_test.copy()
    df_temp['label'] = preds

    for row in df_temp.iterrows():
        cur_profit = 0.0
        profit = row[1]['profit']
        residual = row[1]['residual']
        
        # For each currently held trade in portfolio, adjust freshness and profit with each timestep. 
        for position in trades_data:
            position['fresh'] += 1
            position['profit'] += profit
            # If the residual crosses the threshold or the time runs out, unload the trade. 
            if (position['residual'] - threshold <= residual) or position['fresh'] >= window:
                cur_profit+=position['profit']
                # To keep track of average trade value. 
                trade_profit_timeline.append(position['profit'])
                time_held_timeline.append(position['fresh'])
                trades_data.remove(position)
        profit_timeline.append(cur_profit)
        total_profit+=cur_profit

        if row[1]['label']:
            trades_data.append({'profit': 0, 'residual': residual, 'fresh': 0})
            
    # Appends daily p/l array to array for easier calculations in the future.         
    df_temp['profit_timeline'] = profit_timeline
    
    backtest_results['total_profit'] = total_profit
    backtest_results['daily_profit_timeline'] = profit_timeline
    backtest_results['trade_profit_timeline'] = trade_profit_timeline
    backtest_results['time_held_timeline'] = time_held_timeline
    backtest_results['trades_executed'] = len(trade_profit_timeline)
    backtest_results['params'] = params
    backtest_results['precision'] = precision(df_temp['label'], df_test['label'])
    backtest_results['mean_profit_per_trade'] = np.mean(trade_profit_timeline)
    backtest_results['sharpe'] = sharpe(df_temp)
    backtest_results['sortino'] = sortino(df_temp)
    
    if plot:
        # Plot Total Profit
        min_date = df_temp.TIMESTAMP.min()
        max_date = df_temp.TIMESTAMP.max()
        plt.figure()
        plt.plot(np.cumsum(profit_timeline), color='green')
        plt.xlabel('Time')
        plt.ylabel('P/L')
        plt.xlim([0, df_temp.shape[0]])
        plt.ylim([0, 12000])
        plt.title('Total P/L from %s to %s \n Window=%d, threshold = %0.5f \n Model Parameters: %s' % (min_date, max_date, window, threshold, params))
        plt.grid()
        plt.savefig(save_dir + 'total_pl_' + param_str +'.jpg')

        # Plot Time Held Histogram
        plt.figure()
        plt.hist(time_held_timeline, bins=5, linestyle='solid', edgecolor='black', color='red')
        ax = plt.gca()
        plt.xlabel('Histogram of Minutes held per trade')
        plt.ylabel('Frequency')
        plt.grid()
        plt.xlim([1, window])
        plt.title('Minutes held per trade \n Model Parameters: %s' % params)
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        plt.savefig(save_dir + '/' + param_str +'minutes_held_hist.jpg')
        
        # Plot Minutes Held vs. Profit Scatter Plot
        plt.figure()
        ax = plt.gca()
        plt.scatter(trade_profit_timeline, time_held_timeline)
        plt.xlabel('Profit/Loss')
        plt.ylabel('Minutes Held')
        plt.title('Minutes Held vs. Profit/Loss')
        plt.grid()
        ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
        plt.savefig(save_dir + '/' + param_str +'minutes_held_hist.jpg')
        
        # Average Trade Histogram
        plt.figure()
        ax = plt.gca()
        plt.hist(trade_profit_timeline, bins=1000)
        plt.xlabel('Profit/Loss')
        plt.ylabel('Frequency')
        plt.title('Histogram of Profit/Loss per Trade\n Average Trade: %0.4f\n Median Trade: %0.4f' % (np.mean(trade_profit_timeline), np.median(trade_profit_timeline)))
        plt.axvline(np.median(trade_profit_timeline), label='Median', linestyle='dashed')
        plt.grid()
        ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
        plt.savefig(save_dir + '/' + param_str +'_average_trade_hist.jpg')
        
    return backtest_results

In [6]:
# Particular set of parameters for the SVM model that we want to backtest

params = {'C': 100,
   'cache_size': 2000,
   'class_weight': {0: 0.5, 1: 0.5},
   'gamma': 1,
   'kernel': 'rbf'}

In [7]:
df_pl = create_pl_df(df_gg, df_ggl, info)

P/L Dataframe saved as "df_residual" parameter.


In [8]:
df_pl.shape

(200447, 10)

In [9]:
labels = fit(newparams, info)

NameError: name 'newparams' is not defined

In [ ]:
save_dir = './plots/backtesting/'

In [ ]:
labels

In [ ]:
newlabels.shape

In [ ]:
df_pl.shape

In [ ]:
pl_info = find_profit_loss(df_pl, newlabels, new_params, save_dir=save_dir)

# Results

### Total P/L after 2 years

In [ ]:
print(pl_info['total_profit'])

### Total Trades Executed

In [ ]:
print(pl_info['trades_executed'])

### Sharpe Ratio

In [ ]:
print(pl_info['sharpe'])

# Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Pool

Gather inputs to a list

Generalize Code to work on any dataset

Gather combined outputs of list to aggregate into total p/l. 

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

In [ ]:
def create_pl_df(df_ticker1, df_ticker2, info, datasets):
    """
    Creates a comprehensive DataFrame by combining the ticker data from the given tickers 
    with the residual information calculated in the OU class. 

    Saves this DataFrame to self.df_residual. 
    
    ## TODO: Rewrite GOOG/GOOGL Labels to make more abstract. 

    :df_ticker1: Ticker data from first asset. 
    :df_ticker2: Ticker data from second asset. 
    :info: Fold dictionary calculated from OU class. 

    """
    ticker1_name = df_ticker1['TICKER']
    ticker2_name = df_ticker2['TICKER']
    
#     df_temp = pd.DataFrame()
#     df_temp_labels = pd.Series()
    for i in info:
        idx = i['test']['index']
        residuals = i['test']['residuals_transform_price']
        beta = i['train']['beta_fit_price']
        df_temp = pd.concat([df_ticker1.loc[idx]['CLOSE'], 
                             beta * df_ticker2.loc[idx]['CLOSE'], 
                             df_ticker1.loc[idx]['price'], 
                             beta * df_ticker2.loc[idx]['price'], 
                             i['test']['residuals_transform_price'], 
                             df_ticker1.loc[idx]['TIMESTAMP']
                            ], axis=1)
        datasets.append(df_temp)
        df_temp['label'] = i['test']['labels']
        df_temp.columns=['GOOG', 'beta*GOOGL', 'GOOG_return', 
                              'beta*GOOGL_return', 'residual', 'TIMESTAMP', 'label']
        # Find Profit of last trade given percent returns. 
        # (y-(1-deltaY)y) - (x-(1-deltaX)x)
        df_temp['beta*GOOGL_gains'] = df_temp['beta*GOOGL']-(1-df_temp['beta*GOOGL_return'])*df_temp['beta*GOOGL']
        df_temp['GOOG_gains'] = df_temp['GOOG']-(1-df_temp['GOOG_return'])*df_temp['GOOG']
        df_temp['profit'] = df_temp['beta*GOOGL_gains'] - df_temp['GOOG_gains']
        df_temp['TIMESTAMP'] = pd.to_datetime(df_temp.TIMESTAMP)
#         df_temp = df_temp.append(df_temp)
#         df_temp_labels = df_temp_labels.append(i['test']['labels'])
    
    return df_temp

In [ ]:
prec_recs = pd.DataFrame()
for i in range(len(info)):
    precs = results.cv_results_['split%d_test_precision'% i]
    x = prec_recs.append(pd.DataFrame(precs).T)
    prec_recs = x
prec_recs = prec_recs.reset_index(drop=True)

avg_precs = []
for col in prec_recs:
    a = prec_recs[col]
    b = a[a> 0]
    avg_precs.append(np.mean(b))
    plt.figure()
    plt.boxplot(a)
    plt.hist(a, bins=10)
    plt.xlim([0, 1])
    plt.ylim([0, 1700])
    plt.title(results.cv_results_['params'][int(col)])
    plt.grid()
    

In [ ]:
best = np.argsort(avg_precs)

In [ ]:
def fit_single(params, train, labels, test, lst):
        b_svm = svm.SVC(**params)
        b_svm.fit(train, labels)
        pred = b_svm.predict(test)
        lst.append(pred)

def fit(params, info):
    pool = mp.Pool(processes=16)
    manager = mp.Manager()
    preds = manager.list()
            
    start=timer()
    stuff = [(params, i['train']['df_scale'], i['train']['labels'], i['test']['df_scale'], preds) for i in info]
    [pool.apply(fit_single, i) for i in stuff]
    end=timer()
    pool.close()
    pool.join()
    print("Time Elapsed: %0.2f" % (end-start))

    return np.hstack(np.array(preds))

In [ ]:
x = []
for idx in best:
    param = results.cv_results_['params'][idx]
    labels = fit(param, info)
    pl_info = find_profit_loss(df_pl, labels, param, save_dir=save_dir, plot=False)
    x.append(pl_info)
    print('Finished %d' % idx)

In [ ]:
with open(save_dir+'backtest_results_129.pkl', 'wb') as f:
    pickle.dump(x, f)